In [1]:
!pip install -q openai
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

# Method 1 - openai library 이용
## role of the messages author

OpenAI API에서 각 메시지의 저자(`author`) 역할은 다음과 같은 의미를 가집니다:

1. **system**: 시스템 관련 메시지나 지침. 예를 들면, 어시스턴트에게 특정 작업을 수행하는 방법에 대한 지침을 제공하는 경우에 사용됩니다. 이러한 지침은 어시스턴트의 응답을 안내하거나 제한할 수 있습니다.

2. **user**: 사용자가 입력한 메시지. 대체로 사용자의 질문, 요청, 지침 등을 나타냅니다.

3. **assistant**: 어시스턴트(즉, 모델)가 반환하는 응답 또는 메시지. 사용자의 질문에 대한 답변, 제안, 설명 등을 포함할 수 있습니다.

4. **function**: API의 이전 버전에서 사용됐던 것으로, 특정 기능이나 동작을 어시스턴트에게 요청할 때 사용되었습니다. 최신 버전에서는 일반적으로 사용되지 않습니다.

이러한 역할은 대화형 세션에서 메시지의 발신자와 의도를 구별하는 데 도움을 줍니다. API는 이 정보를 사용하여 적절한 방식으로 반응하거나 응답합니다.

In [6]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

This is a test.

In [5]:
# assistant 에 대한 지침이 되는 context 정보를 제공
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
]

while True:
    # 사용자로부터 메시지 입력 받기
    message = input("User : ")

    # 사용자가 'stop'이라고 입력하면 대화 종료
    if message.lower() == "stop":
        break
    else:
        # 사용자의 메시지를 메시지 목록에 추가
        messages.append(
             {"role": "user", "content": message}
        )

        # OpenAI API를 사용하여 대화형 완성 진행
        completion = openai.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=messages
        )

    # 모델의 답변 추출
    reply = completion.choices[0].message.content

    # 모델의 답변 출력
    print(f"ChatGPT: {reply}")

    # 모델의 답변을 메시지 목록에 추가
    messages.append({"role": "assistant", "content": reply})

User : 너는 누구야
ChatGPT: 저는 인공지능 기반의 도움을 주는 개인 비서입니다. 어떤 도움이 필요하신가요?
User : 강남에서 남산에 가는 길을 알려줘
ChatGPT: 강남에서 남산으로 가는 길은 여러 가지 방법이 있습니다. 가장 편리한 방법은 지하철을 이용하는 것인데요, 다음은 그 방법에 대한 안내입니다.

1. 강남역 (2호선)에서 노량진 방향으로 지하철을 탑니다.
2. 3개 역을 지나 압구정역에 도착한 뒤, 6번 출구로 나옵니다.
3. 압구정역에서 남산 방향으로 택시를 타거나 도보로 이동합니다. 걸어서는 약 30분 정도 소요됩니다.

다른 대안으로는 버스를 이용하는 것이 있습니다. 강남역 근처 버스 정류장에서 버스를 타고 남산으로 이동할 수 있습니다. 정확한 버스 번호와 정류장을 알려드리려면 출발 지점과 도착 지점의 구체적인 위치를 알려주세요.

또한, 남산에는 주차 공간이 제한되어 있으니 차량을 이용하실 경우 사전에 주변 주차장 예약과 관련된 사항도 고려해야 합니다.

도움이 되셨나요? 추가적인 도움이 필요하시면 언제든지 말씀해주세요.
User : quit
ChatGPT: 알겠습니다. 궁금한 점이 있으면 언제든지 도움을 요청해주세요. 안녕히 가세요!
User : stop


In [ ]:
messages

# Method 2 - API Endpoint 이용

payload"는 주로 HTTP 요청을 보낼 때 전달되는 데이터를 의미합니다. API 호출에서 "payload"는 주로 POST 요청의 본문에 포함된 데이터를 나타냅니다.  

- temperature : 사용할 샘플링 온도는 0에서 2 사이입니다. 0.8과 같이 값이 높을수록 출력이 더 무작위로 생성(Hallucination 발생 가능)되고, 0.2와 같이 값이 낮을수록 더 집중적이고 결정적이게 됩니다.

- top_p : temperature의 대안으로, 확률 top_p인 토큰의 결과를 고려합니다. 따라서 0.1은 상위 10% 확률을 구성하는 토큰만 고려된다는 의미입니다.

**temperature 와 top_p 중 하나만 변경하고, 둘 다 변경하지는 않는 것이 좋습니다.**  

- n:  각 입력 메시지에 대해 생성할 chat completion 선택 수입니다.  

-  presence_penalty : -2.0과 2.0 사이의 숫자입니다. 양수 값은 지금까지 텍스트에 나타나는지 여부에 따라 새 토큰에 불이익을 주어 모델이 새로운 주제에 관해 이야기할 가능성을 높입니다.

- frequency_penalty : -2.0과 2.0 사이의 숫자입니다. 양수 값은 지금까지 텍스트의 기존 빈도를 기반으로 새 토큰에 불이익을 주어 모델이 동일한 줄을 그대로 반복할 가능성을 줄입니다.  

Bearer는 HTTP 인증 스키마 중 하나입니다. 웹 서비스에서 사용하는 토큰 기반의 인증 방식 중 가장 일반적인 방식입니다. Bearer를 사용하면, 클라이언트는 해당 토큰을 포함하여 서버에 요청을 보낼 수 있으며, 서버는 이 토큰을 검증하여 해당 클라이언트의 요청을 인증합니다. Bearer {토큰} 형식에서 {토큰} 부분은 실제 API를 사용하기 위한 인증 토큰을 나타냅니다.

In [7]:
import requests

# OpenAI API의 URL을 정의
URL = "https://api.openai.com/v1/chat/completions"

# 요청에 필요한 데이터를 payload 변수에 저장
payload = {
    "model": "gpt-3.5-turbo",
    "messages": [{"role": "user", "content": f"세계 바둑 Champion이 누구야 ?"}],
    "temperature": 1.0,
    "top_p": 1.0,
    "n": 1,
    "stream": False,     # 스트림모드 사용 여부
    "presence_penalty": 0,  # 결과의 일관성에 영향
    "frequency_penalty": 0,  # 단어의 사용 빈도에 영향
}

# 요청 헤더에 내용 유형 및 인증 키 설정
headers = {
    "Content-Type": "application/json",  # 요청 본문의 유형을 JSON으로 지정
    "Authorization": f"Bearer {openai.api_key}"  # 인증 키 포함
}

# requests.post를 사용하여 OpenAI API에 POST 요청을 보냅니다.
response = requests.post(URL, headers=headers, json=payload, stream=False)

In [8]:
response

<Response [200]>

### response 내용
1. **choices**: API 응답의 주요 내용이 포함된 배열입니다.
   - **finish_reason**: 응답이 완료된 이유입니다. 'stop'은 출력이 종료된 이유가 됩니다.
   - **index**: 선택 사항의 인덱스 번호입니다.
   - **message**: 사용자 또는 보조프로그램의 메시지 내용입니다.
     - **content**: 보조프로그램이 생성한 메시지 내용입니다.
     - **role**: 이 메시지를 생성한 주체입니다.  'assistant'는 메시지가 OpenAI의 assistena program으로부터 생성되었다는 것을 의미합니다.

2. **created**: 응답이 생성된 시간의 타임스탬프입니다.

3. **id**: 응답에 할당된 고유 식별자입니다.

4. **model**: 사용된 모델의 이름입니다.

5. **object**: 이 객체의 유형을 나타냅니다. 'chat.completion'은 채팅 완료 응답임을 나타냅니다.

6. **usage**: 이 요청에서 사용된 토큰의 수를 나타내는 정보입니다.
   - **completion_tokens**: 응답 내용에서 사용된 토큰 수입니다.
   - **prompt_tokens**: 사용자의 원래 질문에서 사용된 토큰 수입니다.
   - **total_tokens**: 전체 토큰 수입니다.

In [9]:
import pprint
import json

# 응답 내용을 JSON으로 파싱
response_json = response.json()

# 파싱된 JSON 내용을 pretty-print로 출력
pprint.pprint(response_json)

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '현재 세계 바둑 챔피언은 알파고(AI 프로그램) 입니다. 그 이전에는 '
                                     '이세돌 9단과 이란의 압도적인 강자인 알리 자데 모라디아바디가 세계 바둑 '
                                     '챔피언으로 알려져 있습니다.',
                          'role': 'assistant'}}],
 'created': 1704932718,
 'id': 'chatcmpl-8fdAkomElPoJt1rNobGftva6jpL9g',
 'model': 'gpt-3.5-turbo-0613',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'usage': {'completion_tokens': 100, 'prompt_tokens': 22, 'total_tokens': 122}}


In [10]:
print(response_json['choices'][0]['message']['content'])

현재 세계 바둑 챔피언은 알파고(AI 프로그램) 입니다. 그 이전에는 이세돌 9단과 이란의 압도적인 강자인 알리 자데 모라디아바디가 세계 바둑 챔피언으로 알려져 있습니다.
